# MNIST: CNN
Solving the MNIST problem using Convulutional Neural Network (CNN) instead of Linear

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("data/MNIST/",one_hot=True)